In [1]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib inline
#inline

# Importamos las librerías necesarias
import numpy as np
import time
import cv2 as cv
import matplotlib.pyplot as plt
import os

In [2]:
root=r'C:\Users\Aulamultimedia\MEGA\Especializacion en Inteligencia Artificial UBA\Materias\Bimestre 2\Vision por Computadora 1\TP7_video.avi'
cap = cv.VideoCapture(root)
if not cap.isOpened:
    print('Falla al abrir el archivo: ' + filename)


In [3]:
# se define la función para capturar el fondo mediante el estimador de Mediana
def fondo_extract(frames):
    fondo_Mediana = np.median(frames, axis=0).astype(dtype=np.uint8) 
    return fondo_Mediana

In [10]:
# Implementamos función para quedarnos con los primeros N frames del video
def firstN(video, n):
    frameIds = np.arange(n)
    frames = []
    for fid in frameIds:
        video.set(cv.CAP_PROP_POS_FRAMES, fid)
        ret, frame = video.read()
        frames.append(frame)
        
        #Se calcula mediana de frames
        fmedian = fondo_extract(frames)
        # Escribimos sobre la imagen el número de frame procesado
        cv.rectangle(fmedian, (10, 2), (100,20), (255,255,255), -1)
        cv.putText(fmedian, str(cap.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
        #Se muestra el frame calculado con mediana
        cv.imshow('Median frame', fmedian)
        cv.waitKey(1)
    cv.destroyAllWindows() 
     
    fmedian_gray = cv.cvtColor(fmedian, cv.COLOR_BGR2GRAY)
    return fmedian, fmedian_gray




In [15]:
# Vemos como funciona el filtrado de fondo con el estimador de Mediana
fmedian, fmedian_gray = firstN(cap,60)


In [17]:
# Aplicamos el filtrado por Gausiana
# se define la variable para caputrar el fondo mediante MOG2
backSubGauss = cv.createBackgroundSubtractorMOG2()

# Abrimos el archivo
#-------------------
capture = cv.VideoCapture(root)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
    fgMask = backSubGauss.apply(frame)
    
    # Aplicamos el algoritmo de filtrado de fondo con el estimador de Mediana
    # Primero convertimos la imagen original a escala de grises
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Calculamos diferencia entre la original y nuestro fondo en escala de grises
    fmedian_gray_sinfondo = cv.absdiff(frame, fmedian_gray)
    th, fmMask = cv.threshold(fmedian_gray_sinfondo, 30, 255, cv.THRESH_BINARY)
    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(fmMask, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(fmMask, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))

    cv.rectangle(fgMask, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(fgMask, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # mostramos imagen sin fondo aplicando algoritmo gausiano y también por la Mediana: 
    cv.imshow('MOG2', fgMask)
    cv.imshow('Median', fmMask)
    
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

## Se observa que el filtrado por Mediana converge mas rápido y tiene menos ruido. A su vez, en esta implementación consume muchos mas recursos computacionales.